### 동적수화 번역 모델

In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from PIL import ImageFont, ImageDraw, Image

def meadia_pipe(model, actions):
    seq_length = 30

    Text = [''] # 모션 텍스트
    
    # MediaPipe hands model
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(0)

    seq = []
    action_seq = []

    while cap.isOpened():
        ret, img = cap.read()
        img0 = img.copy()

        img = cv2.flip(img, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if result.multi_hand_landmarks is not None:
            hand_arr = []
            for res in result.multi_hand_landmarks:
                joint = np.zeros((21, 4))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                # Compute angles between joints
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                v = v2 - v1 # [20, 3]
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                # Get angle using arcos of dot product
                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                    v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                angle = np.degrees(angle) # Convert radian to degree

                d = np.concatenate([joint.flatten(), angle])
                hand_arr.extend(d)

                mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(hand_arr) == 99:
                    hand_arr.extend(np.zeros(99))

            if len(hand_arr) > 198:
                continue

            seq.append(hand_arr)

            if len(seq) < seq_length:
                continue

            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
            y_pred = model.predict(input_data).squeeze()
            i_pred = int(np.argmax(y_pred))
            conf = y_pred[i_pred]
            if conf < 0.9:
                continue

            #print(i_pred)
            action = actions[i_pred]
            action_seq.append(action)
            if len(action_seq) < 3:
                continue
            this_action = '?'
            if action_seq[-1] == action_seq[-2] == action_seq[-3]:
                this_action = action
            font = ImageFont.truetype("fonts/gulim.ttc", 20)
            img = Image.fromarray(img)
            draw = ImageDraw.Draw(img)
            draw.text((30,50), this_action, font=font, fill=(0,0,255))
            img = np.array(img)

            if Text[-1] != this_action :
                Text.append(this_action)

        # out.write(img0)
        # out2.write(img)
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            
            return Text

### 단어to문장 모델

In [27]:
import numpy as np
import json
from tensorflow import keras

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)     

def predict_mo(lst):
    
    reconstructed_model = keras.models.load_model("WTS_model.h5")
    char_indices = json.load(open("char_indices.json","r"))
    indices_char = json.load(open("indices_char.json","r"))   
    
    result =''
    
    for i in lst:
        if i[-1] == '요' or i[-1] == '다':
            result += i + ' '
            continue
            
        x = np.zeros((1, 5, len(char_indices)))
        x[0, len(i), char_indices[i]] = 1.

        preds = reconstructed_model.predict(x, verbose=0)[0]
        next_index = sample(preds)
        next_char = indices_char[str(next_index)]
        
        
        #y = josa_fuc(i, next_char)
        result += (i + next_char + ' ')
        
    return result

def new_text(text):
    try:
        text.remove('')
        while '?' in text:
            text.remove('?')
    except:
        return text
    return text

In [30]:
# mp_words = ['', '?', '오늘', '?', '날씨', '?', '맑다']

# np_words2 = new_text(mp_words)
# print(np_words2)

# sentence1 = predict_mo(np_words2)
# print(sentence1)

### 파이프라인 테스트

In [29]:
actions1 = [
    '오늘',
    '날씨',
    '맑다',
]
model1 = keras.models.load_model("mediapipe_model.h5")

mp_words = meadia_pipe(model1, actions1)
    
#print(mp_words) 
np_words2 = new_text(mp_words)
#print(np_words2)

sentence1 = predict_mo(np_words2)
print(sentence1)  

오늘은 날씨가 맑다 
